In [30]:

import torch
import cv2
from matplotlib import pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import os
import random
import json
from PIL import Image
from tqdm import tqdm
from scipy.spatial.transform import Rotation as Rot_mat

In [31]:
'''Data Adress'''
# unityeyes_folder = '/home/daiki/HDD_1/sharefolder/imgs/'
# unityeyes_folder = '/home/daiki/HDD_1/code/unityeyes_preprocess_0.0.0/imgs/'
# unityeyes_folder = '/home/daiki/HDD_1/sharefolder/unityeyes/save_imgs/'
unityeyes_folder = '/home/daiki/HDD_1/sharefolder/Images/unityeyes/save_rot_imgs/'
save_img_path = '/home/daiki/HDD_1/data/eval_deformation_1000/'
save_json_path = '/home/daiki/HDD_1/data/eval_deformation_1000_json/'
# unityeyes_folder = '/home/daiki/HDD_1/sharefolder/unityeyes/imgs'
# save_img_path = '/home/daiki/HDD_1/code/unityeyes_preprocess_0.0.0/confirm_eyeballcenter/'
# save_json_path = '/home/daiki/HDD_1/code/unityeyes_preprocess_0.0.0/confirm_eyeballcenter/'

'''Make Directory'''
os.makedirs(save_img_path, exist_ok = True)
os.makedirs(save_json_path, exist_ok = True)


In [32]:
def img_trans_coord(x, y, xl, xr, yd, yu):

    trans_ratio_x = random.uniform(xl, xr)
    trans_ratio_y = random.uniform(yd, yu)
    trans_coord_x = int(x - (trans_ratio_x * x))
    trans_coord_y = int(y - (trans_ratio_y * y))
    
    return trans_coord_x, trans_coord_y

In [33]:
def img_random_crop(img, x, y, size_x, size_y):
    # x = img_size_x = 60 * 3
    # y = img_size_y = 36 * 3
    croped_img = img[y-int(size_y/2):y+int(size_y/2),x-int(size_x/2):x+int(size_x/2),:]
    
    return croped_img

In [34]:
def rotM(deg_x, deg_y, deg_z):
    Rx = np.array([[1, 0, 0],
                   [0, np.cos(deg_x), np.sin(deg_x)],
                   [0, -np.sin(deg_x), np.cos(deg_x)]])
    
    Ry = np.array([[np.cos(deg_y), 0, -np.sin(deg_y)],
                   [0, 1, 0],
                   [np.sin(deg_y), 0, np.cos(deg_y)]])
    
    Rz = np.array([[np.cos(deg_z), np.sin(deg_z), 0],
                   [-np.sin(deg_z), np.cos(deg_z), 0],
                   [0, 0, 1]])

    
    R = Rz.dot(Ry).dot(Rx)
    
    return R
    

In [35]:
def gaze_data_spliter(gd_list):
    gaze_datas = gd_list.strip('()')
    gaze_datas = gaze_datas.split(",")
    g_x = gaze_datas[0]
    g_y = gaze_datas[1]
    g_z = gaze_datas[2]
    gaze_array = np.array([float(g_x), float(g_y), float(g_z)])
    
#     return g_x, g_y, g_z
    return gaze_array

In [36]:
def head_data_spliter(head_list):
    head_datas = head_list.strip('()')
    head_datas = head_datas.split(',')
    head_x = head_datas[0]
    head_y = head_datas[1]
    head_z = head_datas[2]

    return head_x, head_y, head_z

In [37]:
def show_img(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img)
    plt.show()

In [38]:
def img_down_size(img, d_size_ratio):
    img = cv2.resize(img, dsize=None, fx=d_size_ratio, fy=d_size_ratio)
    
    return img

In [39]:
def concate_list(eyelids, iris, gaze):
    output_list = [eyelids, iris, gaze]
    
    return output_list

In [40]:
def data_forming(array):
    data_list = []
        
    for datas in array:
        data = (round(datas[0],4), round(datas[1],4), 0.0000)
        data_list.append(str(data))
    
    return data_list

In [41]:
def data_forming_eyeball(eb_coord):
    data_list = []
    eb_coord = np.round(eb_coord, decimals=4)
    data = (eb_coord[0], eb_coord[1])
    data_list.append(str(data))
    
    return data_list

In [42]:
def data_forming_gaze(gaze):
    data_list = []
    data = (round(gaze[0],4), round(gaze[1],4), round(gaze[2],4), 0.0000)
    data = str(data)
    return data

In [43]:
def random_down_size(img, min_coef, max_coef):
    random_coef = random.uniform(min_coef,max_coef)
    img = img_down_size(img, random_coef)
    return img, random_coef

In [44]:
def pre_process(data_list):
    output_list = []
    
    for i in range(len(data_list)):
        data = data_list[i].strip('()')
        data = data.split(",")
        output_data = [float(data[0]), float(data[1])]
        output_list.append(output_data)
        
    return output_list

In [45]:
def pre_process_gaze(data_list):
    output_list = []
    data = data_list.strip('()')
    data = data.split(",")
    output_data = [float(data[0]), float(data[1])*-1, float(data[2])]
        
    return output_data

In [46]:
def draw_circle_image(img, eyelids_array, iris_array, caruncle_array, eb_center):
    circle_img = img.copy()
    
    for i in range(len(eyelids_array)):
        eyelids_x = eyelids_array[i][0]
        eyelids_y = eyelids_array[i][1]
        circle_img = cv2.circle(circle_img, (int(eyelids_x), img.shape[0]-int(eyelids_y)), 1, (0,0,255),thickness = -1)
    
    for j in range(len(iris_array)):
        iris_x = iris_array[j][0]
        iris_y = iris_array[j][1]
        circle_img = cv2.circle(circle_img, (int(iris_x), img.shape[0]-int(iris_y)), 1, (255,0,0),thickness = -1)
        
    for k in range(len(caruncle_array)):
        caruncle_x = caruncle_array[k][0]
        caruncle_y = caruncle_array[k][1]
        circle_img = cv2.circle(circle_img, (int(caruncle_x), img.shape[0]-int(caruncle_y)), 1, (0,255,0),thickness = -1)
    
    circle_img = cv2.circle(circle_img, (int(eb_center[1]), int(img.shape[0] - eb_center[0])), 1, (255,255,255), thickness = 1)
        
        
    show_img(circle_img)
#     import pdb;pdb.set_trace()

In [47]:
def move_ldmk_random_dwsize(eyelids, caruncle, iris, eyeball_center, coef, deg, mat):
    eyelids = np.array(eyelids) * coef
    caruncle = np.array(caruncle) * coef
    iris = np.array(iris) * coef
    eyeball_center = np.array(eyeball_center) * coef
    
    return eyelids, caruncle, iris, eyeball_center
    

In [48]:
def ldmk_reverse(data_list, img_h):
    coord_list = []
    
    for i in range(len(data_list)):
        x_coord = float(data_list[i][0])
#         y_coord = (img_h - float(data_list[i][1]))
        y_coord = float(data_list[i][1])
        rev_x_coord, rev_y_coord = int(x_coord), int(y_coord)
        coord_list.append([rev_x_coord, rev_y_coord])
    
    return coord_list

In [49]:
def rotate_image(deg, img):

    w = img.shape[1]
    h = img.shape[0]

    mat = cv2.getRotationMatrix2D((w/2, h/2), deg, 1.0)
    affine_img = cv2.warpAffine(img, mat, (w, h))
    
#     show_img(affine_img)
    
    return affine_img, mat
    

In [50]:
def rot_ldmk(deg, ldmk_mtrx, x, y):
    
    ldmk_list = []
#     print("ldmk_mtrx.shape:",np.array(ldmk_mtrx).shape)
    
    rad = np.deg2rad(deg)
#     print("rad:",rad," deg:",deg)
    
    for i in range(len(ldmk_mtrx)):
#         print("ldmk_mtrx[i][0] :",ldmk_mtrx[i][0])
#         print("ldmk_mtrx[i][1] :",ldmk_mtrx[i][1])
#         print("x:",x," y:",y)
        ldmk_x = ldmk_mtrx[i][0] - (x/2)
        ldmk_y = ldmk_mtrx[i][1] - (y/2)


        ldmk_array = np.array([ldmk_x, ldmk_y])
#         print("ldmk_array.shape:",ldmk_array.shape)
        
        R = [[np.cos(rad), -np.sin(rad)],
             [np.sin(rad), np.cos(rad)]]
        
        rotated_ldmk = np.dot(R, ldmk_array)
        rotated_ldmk[0] = rotated_ldmk[0] + (x/2)
        rotated_ldmk[1] = rotated_ldmk[1] + (y/2)

        ldmk_list.append(rotated_ldmk)
    ldmk_out_array = np.array(ldmk_list)
    
    return ldmk_out_array

In [51]:
def rot_gaze(deg, gaze):
    gaze_array = np.array([gaze[0], gaze[1], gaze[2]])
#     print(gaze_array)
    r = Rot_mat.from_euler('z', deg * -1, degrees=True)
    rotated_gaze_array = r.apply(gaze_array)

    return rotated_gaze_array

In [52]:
# def move_ldmk_random_crop(moved_img, original_img, data_list, moved_x, moved_y):
#     half_w = moved_img.shape[1]/2
#     half_h = moved_img.shape[0]/2
#     original_img_width =  original_img.shape[0]
#     circle_img = moved_img.copy()
#     coord_list = []

#     for i in range(len(data_list)):
#         # print('half_w,data_list[i][0]), moved_x:',half_w, float(data_list[i][0]), moved_x)
#         moved_x_coord = half_w + float(data_list[i][0]) - moved_x
# #         moved_y_coord = half_h + (original_img_width - float(data_list[i][1]) - moved_y)
#         moved_y_coord = half_h + float(data_list[i][1]) - moved_y
#         draw_x_coord, draw_y_coord = int(moved_x_coord), int(moved_y_coord)
#         coord_list.append([moved_x_coord, moved_y_coord])

#         '''Draw Landmark'''
#         circle_img = cv2.circle(circle_img, (draw_x_coord, draw_y_coord), 3, (255,0,0),thickness = -1)

# #     show_img(circle_img)
#     return coord_list

In [53]:
def move_ldmk_random_crop(moved_img, data_list, moved_x, moved_y):
    coord_list = []
    circle_img = moved_img.copy()

    for i in range(len(data_list)):
        # print('half_w,data_list[i][0]), moved_x:',half_w, float(data_list[i][0]), moved_x)
        moved_x_coord = float(data_list[i][0]) - moved_x
#         moved_y_coord = half_h + (original_img_width - float(data_list[i][1]) - moved_y)
        moved_y_coord = float(data_list[i][1]) - moved_y
        draw_x_coord, draw_y_coord = int(moved_x_coord), int(moved_y_coord)
        coord_list.append([moved_x_coord, moved_y_coord])

        '''Draw Landmark'''
        circle_img = cv2.circle(circle_img, (draw_x_coord, draw_y_coord), 3, (255,0,0),thickness = -1)

    # show_img(circle_img)
    return coord_list

In [54]:
# def move_eyeball_center_random_crop(moved_img, original_img, eyeball_center, moved_x, moved_y):
#     half_w = moved_img.shape[1]/2
#     half_h = moved_img.shape[0]/2
#     original_img_width =  original_img.shape[0]
#     circle_img = moved_img.copy()
#     coord_list = []
#     print(eyeball_center, moved_x, moved_y)
    
#     moved_x_coord = half_w + float(eyeball_center[1]) - moved_x
#     moved_y_coord = half_h + float(eyeball_center[0]) - moved_y
#     draw_x_coord, draw_y_coord = int(moved_x_coord), int(moved_y_coord)
#     eyeball_center = [moved_y_coord, moved_x_coord]
# #     print(eyeball_center)

#     circle_img = cv2.circle(circle_img, (draw_x_coord, draw_y_coord), 3, (255,0,0),thickness = -1)
# #     show_img(circle_img)
#     return eyeball_center

In [55]:
def move_eyeball_center_random_crop(moved_img, eyeball_center, moved_x, moved_y):
    coord_list = []
    circle_img = moved_img.copy()
    
    moved_x_coord = float(eyeball_center[1]) - moved_x
    moved_y_coord = float(eyeball_center[0]) - moved_y
    draw_x_coord, draw_y_coord = int(moved_x_coord), int(moved_y_coord)
    eyeball_center = [moved_y_coord, moved_x_coord]
#     print(eyeball_center)

    circle_img = cv2.circle(circle_img, (draw_x_coord, draw_y_coord), 3, (255,0,0),thickness = -1)
#     show_img(circle_img)
    return eyeball_center

In [56]:
def show_gaze_img(img, gaze, iris, eb_center):
    iris_center = np.mean(iris, axis=0)
#     print(iris_center)
    x = int(iris_center[0])
    y = img.shape[0]-int(iris_center[1])
    img = cv2.circle(img, (x, y), 1, (0,0,255), thickness = 1)
    img = cv2.circle(img, (int(eb_center[1]), img.shape[0] - int(eb_center[0])), 1, (255,255,255), thickness = 1)
    img = cv2.arrowedLine(img, (x, y), (x + int(500*gaze[0]), y + int(500*gaze[1])), (255, 0, 0), thickness=1)
    show_img(img)
    
#     import pdb;pdb.set_trace()

In [57]:
def draw_center_line(img, eb_center):
    img = cv2.line(img, (int(img.shape[1]/2), 0), (int(img.shape[1]/2), int(img.shape[0])), (0, 255, 0), thickness=1)
    img = cv2.line(img, (0, int(img.shape[0]/2)), (img.shape[1], int(img.shape[0]/2)), (0, 255, 0), thickness=1)
    # print(eb_center)
    img = cv2.circle(img, (int(eb_center[1]), img.shape[0] - int(eb_center[0])), 1, (0,0,255), thickness = 1)
    show_img(img)

In [59]:
#Unity Eyes Image Size 480 * 600
    #camera parameter : 0,0,0,0
#eye parameters : 0,0,30,30

'''Crop Size'''
img_size_x = 60 * 3
img_size_y = 36 * 3

'''Down size coefficient'''
d_size_coef = 1/3

'''Coefficient of Random Downsize'''
min_coef = 0.5
max_coef = 1.0

'''Coeficinet of Move Coordinate '''
xl = -0.075
xr = 0.085
yd = 0.09
yu = -0.09
# xl = -1
# xr = 1
# yd = -1
# yu = 1

'''Json File key'''
eyelids_key = "interior_margin_2d"
caruncle_key = "caruncle_2d"
iris_key = "iris_2d"
eye_key = "eye_details"
gaze_key = "look_vec"
eye_region_key = "eye_region_details"
head_pose_key = "head_pose"

'''Initialize'''
croped_img_list = []
img_list = os.listdir(unityeyes_folder)
img_num = int(len(img_list) / 2)
j = 1

# for i in tqdm(range(1,img_num)):
for i in tqdm(range(1,1500)):
    img = cv2.imread(os.path.join(unityeyes_folder, "{}.jpg".format(i)))
#     img_h, img_w  = img.shape[0], img.shape[1]
#     print('img_h:',img_h)
#     print('img_w:',img_w)
    img_w = img.shape[1]
    img_h = img.shape[0]
    eyeball_center = [int(img_h/2), int(img_w/2)]
#     print(os.path.join(unityeyes_folder, "{}.jpg".format(i)))

    '''Read json File'''
    with open(os.path.join(unityeyes_folder, "{}.json".format(i))) as json_file:

        json_data_list = json.loads(json_file.read())
        eyelids_data_list = json_data_list[eyelids_key]
        iris_data_list = json_data_list[iris_key]
        gaze_data_list = json_data_list[eye_key][gaze_key]
        head_data_list = json_data_list[head_pose_key]
        caruncle_data_list = json_data_list[caruncle_key]

        '''Pre-Proceess json File'''
        eyelids_data_list = pre_process(eyelids_data_list)
        caruncle_data_list = pre_process(caruncle_data_list)
        iris_data_list = pre_process(iris_data_list)
        gaze_data_list = pre_process_gaze(gaze_data_list)
        # show_gaze_img(img, gaze_data_list, iris_data_list, eyeball_center)
#         print(eyelids_data_list)
        close_eye_checker = abs(eyelids_data_list[4][1] - eyelids_data_list[12][1])
        
        if close_eye_checker >= 30:
        
                ''''Print Moved Landmark'''
                '''Reverse Landmark Y-dim'''
                eyelids_data_list = ldmk_reverse(eyelids_data_list, img.shape[0])
                iris_data_list = ldmk_reverse(iris_data_list, img.shape[0])
                caruncle_data_list = ldmk_reverse(caruncle_data_list, img.shape[0])
                
                '''Show Image'''
                # draw_circle_image(img, eyelids_data_list, iris_data_list, caruncle_data_list, eyeball_center)
                # show_gaze_img(img, gaze_data_list, iris_data_list, eyeball_center)
                
                '''Image Random Rotation'''
                rotate_deg = random.uniform(-15,15)
        #         print("rotate_deg:",rotate_deg)
                rot_img, trans = rotate_image(rotate_deg, img)
        
                '''Rotate Landmark'''
                eyelids_data_list = rot_ldmk(rotate_deg, eyelids_data_list, img_w, img_h)
                caruncle_data_list = rot_ldmk(rotate_deg ,caruncle_data_list, img_w, img_h)
                iris_data_list = rot_ldmk(rotate_deg, iris_data_list, img_w, img_h)      
        #         print(eyelids_data_list)

                '''Rotate Gaze Direction'''
                gaze_data_list = rot_gaze(rotate_deg, gaze_data_list)
        #         import pdb;pdb.set_trace()
                
                ''' Image Random Downsize'''
                downsized_img, random_coef = random_down_size(rot_img, min_coef, max_coef)
                img_h_center = downsized_img.shape[0] / 2
                img_w_center = downsized_img.shape[1] / 2

                '''Image Coord Trans Position'''
                img_transed_xcoord, img_transed_ycoord = img_trans_coord(img_w_center, img_h_center, xl, xr, yd, yu)
                # print('img_transed_xcoord, img_transed_ycoord:',img_transed_xcoord, img_transed_ycoord)

                '''Image Random Crop'''
                croped_img = img_random_crop(downsized_img, img_transed_xcoord, img_transed_ycoord, img_size_x, img_size_y)
                movement_x = (downsized_img.shape[1]/2) - (((downsized_img.shape[1]/2) - img_transed_xcoord) + (img_size_x/2))
                # movement_y = (downsized_img.shape[0]/2) - (((downsized_img.shape[0]/2) - img_transed_ycoord) + (img_size_y/2))
                movement_y = (downsized_img.shape[0]/2) + ((downsized_img.shape[0]/2) - img_transed_ycoord) - (img_size_y/2)
                # print('movement_x,movement_y:',movement_x,movement_y)
                # import pdb;pdb.set_trace()
        #     show_img(croped_img)
                
                ''''Move Landmark'''
                eyelids_data_list, caruncle_data_list, iris_data_list, eyeball_center = move_ldmk_random_dwsize(eyelids_data_list,caruncle_data_list, iris_data_list, eyeball_center, random_coef, rotate_deg, trans)
                
                ''''Print Moved Landmark for down size'''
                # print('downsized_img.shape:',downsized_img.shape)
                # draw_circle_image(downsized_img, eyelids_data_list, iris_data_list, caruncle_data_list, eyeball_center)
                
                # eyelids_list = move_ldmk_random_crop(croped_img, downsized_img, eyelids_data_list, img_transed_xcoord, img_transed_ycoord)
                # caruncle_list = move_ldmk_random_crop(croped_img, downsized_img, caruncle_data_list, img_transed_xcoord, img_transed_ycoord)
                # iris_list = move_ldmk_random_crop(croped_img, downsized_img, iris_data_list, img_transed_xcoord, img_transed_ycoord)
                # eyeball_center = move_eyeball_center_random_crop(croped_img, downsized_img, eyeball_center, img_transed_xcoord, img_transed_ycoord)

                eyelids_list = move_ldmk_random_crop(croped_img, eyelids_data_list, movement_x, movement_y)
                caruncle_list = move_ldmk_random_crop(croped_img, caruncle_data_list, movement_x, movement_y)
                iris_list = move_ldmk_random_crop(croped_img, iris_data_list, movement_x, movement_y)
                eyeball_center = move_eyeball_center_random_crop(croped_img, eyeball_center, movement_x, movement_y)
                # show_gaze_img(croped_img, gaze_data_list, iris_list, eyeball_center)
                
                # draw_circle_image(croped_img, eyelids_list, iris_list, caruncle_list, eyeball_center)

                '''Resize Image'''
                d_sized_img = img_down_size(croped_img, d_size_coef)
                out_put_img = d_sized_img.copy()
                
                '''Move Landmark'''
                d_eyelids = np.array(eyelids_list) * d_size_coef
                d_caruncle = np.array(caruncle_list) * d_size_coef
                d_iris = np.array(iris_list) * d_size_coef
                eyeball_center = np.array(eyeball_center) * d_size_coef

        #         ''''Print Moved Landmark'''
                # draw_circle_image(d_sized_img, d_eyelids, d_iris, d_caruncle, eyeball_center)
                # draw_center_line(out_put_img, eyeball_center)
                # if i % 100 == 0:
                #         show_gaze_img(d_sized_img, gaze_data_list, d_iris, eyeball_center)

                '''Output .json file'''
                out_json_file_name = str(j) +'.json'
                out_img_file_name = str(j) +'.png'
                j += 1
                with open(os.path.join(save_json_path, out_json_file_name), mode='wt', encoding='utf-8') as output_file:
        #             output_data = {eyelids_key:data_coupling(d_eyelids_x, d_eyelids_y), caruncle_key:data_coupling(d_caruncle_x, d_caruncle_y), iris_key:data_coupling(d_iris_x, d_iris_y), eye_key:{gaze_key:gaze_data_list}}
                    output_data = {eyelids_key:data_forming(d_eyelids), caruncle_key:data_forming(d_caruncle), iris_key:data_forming(d_iris), 'eyeball_center':data_forming_eyeball(eyeball_center), eye_key:{gaze_key:data_forming_gaze(gaze_data_list)}}
                    json.dump(output_data, output_file, ensure_ascii=False, indent=2)
                
                '''Transform Grayscale & Save Images'''
                out_put_img = cv2.cvtColor(out_put_img, cv2.COLOR_BGR2GRAY)
                cv2.imwrite(os.path.join(save_img_path, out_img_file_name),out_put_img)
            

        
        
        
        




100%|██████████| 1499/1499 [00:08<00:00, 167.03it/s]
